In [1]:
import numpy as np
import pyautogui
import imutils
import cv2
import objc
from AppKit import NSEvent
import sys
import Quartz
import time
import random
import math
try: 
    import Image
except ImportError:
    from PIL import Image
import pytesseract

import mss
import mss.tools
from PIL import Image
import PIL.ImageOps  
import re
import os
from selenium import webdriver

In [2]:
def open_and_size_browser_window(width, height, x_pos = 0, y_pos = 0, url = 'http://www.slither.io'):

    chrome_options = webdriver.ChromeOptions()
    chrome_options.add_argument("--disable-infobars")

    driver = webdriver.Chrome("./chromedriver", chrome_options=chrome_options)
    driver.set_window_size(width, height)
    driver.set_window_position(x_pos, y_pos)
    driver.get(url)

    return driver

In [3]:
class Mouse():
    down = [Quartz.kCGEventLeftMouseDown, Quartz.kCGEventRightMouseDown, Quartz.kCGEventOtherMouseDown]
    up = [Quartz.kCGEventLeftMouseUp, Quartz.kCGEventRightMouseUp, Quartz.kCGEventOtherMouseUp]
    [LEFT, RIGHT, OTHER] = [0, 1, 2]

    def position(self):
        point = Quartz.CGEventGetLocation( Quartz.CGEventCreate(None) )
        return point.x, point.y

    def location(self):
        loc = NSEvent.mouseLocation()
        return loc.x, Quartz.CGDisplayPixelsHigh(0) - loc.y

    def move(self, x, y):
        moveEvent = Quartz.CGEventCreateMouseEvent(None, Quartz.kCGEventMouseMoved, (x, y), 0)
        Quartz.CGEventPost(Quartz.kCGHIDEventTap, moveEvent)

    def press(self, x, y, button=1):
        event = Quartz.CGEventCreateMouseEvent(None, Mouse.down[button], (x, y), button - 1)
        Quartz.CGEventPost(Quartz.kCGHIDEventTap, event)

    def release(self, x, y, button=1):
        event = Quartz.CGEventCreateMouseEvent(None, Mouse.up[button], (x, y), button - 1)
        Quartz.CGEventPost(Quartz.kCGHIDEventTap, event)

    def click(self, button=LEFT):
        x, y = self.position()
        self.press(x, y, button)
        self.release(x, y, button)

    def click_pos(self, x, y, button=LEFT):
        self.move(x, y)
        self.click(button)

    def to_relative(self, x, y):
        curr_pos = Quartz.CGEventGetLocation( Quartz.CGEventCreate(None) )
        x += current_position.x;
        y += current_position.y;
        return [x, y]

    def move_rel(self, x, y):
        [x, y] = to_relative(x, y)
        moveEvent = Quartz.CGEventCreateMouseEvent(None, Quartz.kCGEventMouseMoved, Quartz.CGPointMake(x, y), 0)
        Quartz.CGEventPost(Quartz.kCGHIDEventTap, moveEvent)

In [4]:
def screenshot(x, y, width, height
               , reduction_factor = 1
               , gray = True
              ):
    
    with mss.mss() as sct:
        # The screen part to capture
        region = {'left': x, 'top': y, 'width': width, 'height': height}

        # Grab the data
        img = sct.grab(region)

        if gray:
            result = cv2.cvtColor(np.array(img), cv2.COLOR_BGRA2GRAY)
        else:
            result = cv2.cvtColor(np.array(img), cv2.COLOR_BGRA2BGR)

        img = result[::reduction_factor, ::reduction_factor]
        return img


def get_score(threshold = 190, var = 10):

    got_it = False
    
    while not got_it:
        img = screenshot(x= score_x, y= score_y, width = score_width, height = score_height, gray = False)

        if pytesseract.image_to_string(img) == 'Skin':
            return -1
        
        for rows in range(0, len(img)):
            for cols in range(0, len(img[0])):
                x = img[rows][cols]
                if x[0] > threshold and x[1] > threshold and x[2] > threshold and max(x) - min(x) < var:
                    img[rows][cols] = [0, 0, 0]
                else:
                    img[rows][cols] = [255,255,255]

        score = pytesseract.image_to_string(img, config='outputbase digits')
        
        score = re.sub("[^0-9]", "", score)
        
        if score != "":            
            score = int(score)
            return score
    

def move_to_radians(radians):
    
    mouse.move(window_center_x + 50 * math.cos(radians)
               , window_center_y + 50 * math.sin(radians))
    
    return radians

def is_dead():
    img = screenshot(x= score_x, y= score_y, width = score_width, height = score_height, gray = False)
    if pytesseract.image_to_string(img) == 'Skin':
        return True
    else: 
        return False


def start_game():
    time.sleep(1)
    mouse.move(start_button_position_x, start_button_position_y)
    pyautogui.click()
    time.sleep(1)
    pyautogui.click()
    time.sleep(1)
    move_to_radians(0)
    
def get_direction():
    x, y = mouse.position
    
    return math.atan2(y, x)


In [5]:
def generate_data(delete_file = False, time_gap = 1):
        
    score = 10
    
    if delete_file:
        if os.path.isfile('data.csv'):
            os.remove('data.csv')
    
    for i in range(0, 50000):
        
        if is_dead():
            print('You are dead.  Starting the game over.')
            start_game()
            score = 10
            time.sleep(1)
        else:
            
            direction = move_to_radians(random.uniform(0, math.pi*2))
            
            data = screenshot(x=window_x
                              , y=window_y
                              , width=window_width
                              , height=window_height
                              , reduction_factor = 5)
            
            values = data.shape[0] * data.shape[1]
        
            data = data.reshape(values)
            
            old_score = score
            
            time.sleep(time_gap)
            score = get_score()
            
            data = np.append(data, [direction, old_score, score, score - old_score])
            
            with open('data.csv', 'ab') as f:
                np.savetxt(f, [data], fmt='%5d', delimiter=',')

            print('Data point collected.  Direction: %0.2f. Score changed by: %i' % (direction, score-old_score))
            


In [7]:
# These are the dimensions of the window that I'm going to use
window_x = 0.0
window_y = 100
width = 600
height = 600
window_width = 600
window_height = 600 - window_y
window_center_x = window_width/2
window_center_y = window_y + window_height/2

# comment these out if you want to consider the whole window
# this is so that you can easily consider less data if you want to
size_of_window = 200.0

window_x = window_center_x - size_of_window/2
window_y = window_center_y - size_of_window/2
window_width = size_of_window
window_height = size_of_window

# Location of the score that is displayed on the screen
score_x = 83
score_y = 580
score_width = 91
score_height = 25

# position of the start button on the screen with these window settings
start_button_position_x = window_center_x
start_button_position_y = 490

# instantiate mouse
mouse = Mouse()

# open slither.io
driver = open_and_size_browser_window(width = width, height = height)

# make some data
generate_data(delete_file = True)

one
You are dead.  Starting the game over.
Data point collected.  Direction: 5.35. Score changed by: 1
Data point collected.  Direction: 4.32. Score changed by: 0
Data point collected.  Direction: 5.96. Score changed by: 0
Data point collected.  Direction: 2.49. Score changed by: 11
Data point collected.  Direction: 4.91. Score changed by: 0
Data point collected.  Direction: 4.96. Score changed by: 0
Data point collected.  Direction: 2.38. Score changed by: 3
Data point collected.  Direction: 2.73. Score changed by: 0
Data point collected.  Direction: 0.81. Score changed by: 2
Data point collected.  Direction: 2.04. Score changed by: 0
Data point collected.  Direction: 3.51. Score changed by: 0
Data point collected.  Direction: 2.23. Score changed by: 0
Data point collected.  Direction: 4.03. Score changed by: 0
Data point collected.  Direction: 5.92. Score changed by: 0
Data point collected.  Direction: 2.15. Score changed by: 0
Data point collected.  Direction: 5.08. Score changed by

KeyboardInterrupt: 